# fuzzymatching

In [2]:
import fuzzymatcher
import pandas as pd 
from pathlib import Path
import re
import numpy as np
import math

In [4]:
#data import

#pairs = pd.read_csv('/Users/maxhdarby/Dropbox/foursquare_location_matching_data/pairs.csv',index_col = False)
#s_s = pd.read_csv('/Users/maxhdarby/Dropbox/foursquare_location_matching_data/sample_submission.csv',index_col = False)
#test = pd.read_csv('/Users/maxhdarby/Dropbox/foursquare_location_matching_data/test.csv',index_col = False)
#train = pd.read_csv('/Users/maxhdarby/Dropbox/foursquare_location_matching_data/train.csv',index_col = False)
pairs = pd.read_csv('/Users/maxhdarby/Dropbox/foursquare_location_matching_data/data/model_training/train_raw.csv', index_col = False)
#print(pairs.head())
print(pairs.info())
#print(train.info())






<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025255 entries, 0 to 1025254
Data columns (total 13 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   id                 1025255 non-null  object 
 1   name               1025254 non-null  object 
 2   latitude           1025255 non-null  float64
 3   longitude          1025255 non-null  float64
 4   address            667882 non-null   object 
 5   city               755557 non-null   object 
 6   state              646335 non-null   object 
 7   zip                488836 non-null   object 
 8   country            1025246 non-null  object 
 9   url                240899 non-null   object 
 10  phone              308411 non-null   object 
 11  categories         936666 non-null   object 
 12  point_of_interest  1025255 non-null  object 
dtypes: float64(2), object(11)
memory usage: 101.7+ MB
None


In [ ]:
#some useful formatting 
#.str.normalize('NFKD')
#.drop_duplicates(subset=['column_name'])



In [20]:
#sample dataset for testing
sub_df = train.head(10)
full_df = train.head(100)

In [21]:
#matching process with fuzzymatcher
#issue with fuzzymatcher is that it only joins one - 
#so how best to manage that? without re-running it multiple times?
# use 'link_table' instead of the fuzzy_left_join
# can set threshold using match_score > chosen_threshold

# Columns to match on from df_left
left_on = ["name"]

# Columns to match on from df_right
right_on = ["name"]

# Note that if left_id_col or right_id_col are admitted a unique id will be autogenerated
output = fuzzymatcher.link_table(sub_df, full_df, left_on, right_on, left_id_col = "id", right_id_col = "id")

print(output)


#output = fuzzymatcher.fuzzy_left_join(sub_df, full_df, left_on="column_name",right_on = "column_name")

#output.to_excel("output_v1.0.xlsx")

#print("matches complete")

           __id_left        __id_right  match_score  match_rank  \
0   E_000001272c6c5d  E_000001272c6c5d     0.473550           1   
1   E_000002eae2a589  E_000002eae2a589     0.326303           1   
2   E_000007f24ebc95  E_000007f24ebc95     0.237095           1   
3   E_000008a8ba4f48  E_000008a8ba4f48     0.163151           1   
4   E_00001d92066153  E_00001d92066153     0.463515           1   
5   E_000023d8f4be44  E_000023d8f4be44     0.326303           1   
6   E_00002a131a2bf6  E_00002a131a2bf6     0.479419           1   
7   E_00002a131a2bf6  E_00002a131a2bf6     0.479419           1   
8   E_00002a131a2bf6  E_00002a131a2bf6     0.479419           1   
9   E_00002a131a2bf6  E_00002a131a2bf6     0.479419           1   
10  E_0000764d65557e  E_0000764d65557e     0.279705           1   
11  E_00007dcd2bb53f  E_00007dcd2bb53f     0.513596           1   

                  name_left               name_right  
0      Café Stad Oudenaarde     Café Stad Oudenaarde  
1            Cario

In [12]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

#fuzzymatching with fuzzywuzzy
print(fuzz.ratio("012480755","021480755"))
#Levehshtein ratio is calculated by dividing the Levenshtein distance by the maximum of the length of the string1 and string 2.

print(fuzz.partial_ratio("Max H Darby","Darby"))
#The output of the code gives 100 as partial_ratio() just checks if either string is a substring of the other

print(fuzz.ratio("ร้านตัดผมการาเกด"," ร้านตัดผม กราาเกด"))

print(fuzz.partial_ratio("ร้านตัดผมการาเกด"," ร้านตัดผมการาเกด"))


        

89
100
88
100


In [23]:
#using fuzzy wuzzy to take one input and go through an dataframe - saves match scores to dataframe
#stop forgetting that the dataframe is already matched
pairs = pairs.fillna('')

from fuzzywuzzy import fuzz
from fuzzywuzzy import process
#len(pairs)
#pairs['name_ratio'] = ''
idx = (pairs.applymap(type) == str).all(0)
pairs_new = pairs[pairs.columns[idx]]

for i in pairs.index:
    if i % 100000 == 0:
        print(i)
    #print(pairs.loc[i,'city_1'],pairs.loc[i,'city_2'])
    pairs.loc[i,'name_ratio'] = fuzz.ratio(pairs.loc[i,'name_1'],pairs.loc[i,'name_2'])
    #pairs.loc[i,'city_ratio'] = fuzz.ratio(pairs.loc[i,'city_1'],pairs.loc[i,'city_2'])
    #pairs.loc[i,'state_ratio'] = fuzz.ratio(pairs.loc[i,'state_1'],pairs.loc[i,'state_2'])
    #pairs.loc[i,'zip_ratio'] = fuzz.ratio(pairs.loc[i,'zip_1'],pairs.loc[i,'zip_2'])
    #pairs.loc[i,'country_ratio'] = fuzz.ratio(pairs.loc[i,'country_1'],pairs.loc[i,'country_2'])
    #pairs.loc[i,'url_ratio'] = fuzz.ratio(pairs.loc[i,'url_1'],pairs.loc[i,'url_2'])
    #pairs.loc[i,'categories_ratio'] = fuzz.ratio(pairs.loc[i,'categories_1'],pairs.loc[i,'categories_2'])
    
    #pairs.loc[i,'name_ratio_part'] = fuzz.partial_ratio(pairs.loc[i,'name_1'],pairs.loc[i,'name_2'])
    #pairs.loc[i,'city_ratio_part'] = fuzz.partial_ratio(pairs.loc[i,'city_1'],pairs.loc[i,'city_2'])
    #pairs.loc[i,'state_ratio_part'] = fuzz.partial_ratio(pairs.loc[i,'state_1'],pairs.loc[i,'state_2'])
    #pairs.loc[i,'zip_ratio_part'] = fuzz.partial_ratio(pairs.loc[i,'zip_1'],pairs.loc[i,'zip_2'])
    #pairs.loc[i,'country_ratio_part'] = fuzz.partial_ratio(pairs.loc[i,'country_1'],pairs.loc[i,'country_2'])
    #pairs.loc[i,'url_ratio_part'] = fuzz.partial_ratio(pairs.loc[i,'url_1'],pairs.loc[i,'url_2'])
    #pairs.loc[i,'categories_ratio_part'] = fuzz.partial_ratio(pairs.loc[i,'categories_1'],pairs.loc[i,'categories_2'])

    
print(pairs.head())

 


0
100000


KeyboardInterrupt: 

In [24]:
(pairs.applymap(type) == str).all(0)

id_1                      True
name_1                    True
latitude_1               False
longitude_1              False
address_1                 True
city_1                    True
state_1                   True
zip_1                     True
country_1                 True
url_1                     True
phone_1                   True
categories_1              True
id_2                      True
name_2                    True
latitude_2               False
longitude_2              False
address_2                 True
city_2                    True
state_2                   True
zip_2                     True
country_2                 True
url_2                     True
phone_2                   True
categories_2              True
match                    False
name_ratio               False
city_ratio               False
state_ratio              False
zip_ratio                False
country_ratio            False
url_ratio                False
categories_ratio         False
name_rat

In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

def fuzzy_match(df):
    df = df.fillna('')
    for i in df.index:
        df.loc[i,'name_ratio'] = fuzz.ratio(df.loc[i,'name_1'],df.loc[i,'name_2'])
        df.loc[i,'city_ratio'] = fuzz.ratio(df.loc[i,'city_1'],df.loc[i,'city_2'])
        df.loc[i,'state_ratio'] = fuzz.ratio(df.loc[i,'state_1'],df.loc[i,'state_2'])
        df.loc[i,'zip_ratio'] = fuzz.ratio(df.loc[i,'zip_1'],df.loc[i,'zip_2'])
        df.loc[i,'country_ratio'] = fuzz.ratio(df.loc[i,'country_1'],df.loc[i,'country_2'])
        df.loc[i,'url_ratio'] = fuzz.ratio(df.loc[i,'url_1'],df.loc[i,'url_2'])
        df.loc[i,'categories_ratio'] = fuzz.ratio(df.loc[i,'categories_1'],df.loc[i,'categories_2'])
    
        df.loc[i,'name_ratio_part'] = fuzz.partial_ratio(df.loc[i,'name_1'],df.loc[i,'name_2'])
        df.loc[i,'city_ratio_part'] = fuzz.partial_ratio(df.loc[i,'city_1'],df.loc[i,'city_2'])
        df.loc[i,'state_ratio_part'] = fuzz.partial_ratio(df.loc[i,'state_1'],df.loc[i,'state_2'])
        df.loc[i,'zip_ratio_part'] = fuzz.partial_ratio(df.loc[i,'zip_1'],df.loc[i,'zip_2'])
        df.loc[i,'country_ratio_part'] = fuzz.partial_ratio(df.loc[i,'country_1'],df.loc[i,'country_2'])
        df.loc[i,'url_ratio_part'] = fuzz.partial_ratio(df.loc[i,'url_1'],df.loc[i,'url_2'])
        df.loc[i,'categories_ratio_part'] = fuzz.partial_ratio(df.loc[i,'categories_1'],df.loc[i,'categories_2'])
    return df

# Distance between points

In [26]:
import geopy.distance

def point_dist(df):
    for i in df.index:
        coord_1 = (df.loc[i,'latitude_1'],df.loc[i,'longitude_1'])
        
        coord_2 = (df.loc[i,'latitude_2'],df.loc[i,'longitude_2'])
        
        df['proximity'] = geopy.distance.geodesic(coord_1,coord_2).km
    return df

In [22]:
print(train.head(1))

                 id                  name   latitude  longitude      address  \
0  E_000001272c6c5d  Café Stad Oudenaarde  50.859975   3.634196  Abdijstraat   

         city            state   zip country  url phone categories  \
0  Nederename  Oost-Vlaanderen  9700      BE  NaN   NaN       Bars   

  point_of_interest  
0  P_677e840bb6fc7e  


In [25]:
geo_df = train.head(2)
coord_1 = (geo_df.loc[0,'latitude'],geo_df.loc[0,'longitude'])
print(coord_1)
coord_2 = (geo_df.loc[1,'latitude'],geo_df.loc[1,'longitude'])
print(coord_2)

(50.85997533509319, 3.6341963487748346)
(-22.907225306597013, -43.17824390023254)


In [28]:
print('distance between point: ',geopy.distance.geodesic(coord_1,coord_2).km)

distance between point:  9370.64505095926


# Check same language

In [35]:

from langdetect import detect, DetectorFactory, detect_langs
DetectorFactory.seed = 0
# def detect_lang(text):
#     return detect(text)

# name_eda['name'].apply(detect_lang)

print(detect('ร้านตัดผมการาเกด'))
print(detect('ビックカメラ JR京都駅店'))
print(detect('Hello Max'))
print(detect('i am the automation'))
print(detect("Otec matka syn."))
print()


print(detect_langs('ร้านตัดผมการาเกด'))
print(detect_langs('ビックカメラ JR京都駅店'))
print(detect_langs('hello max'))
print(detect_langs('i am the automation'))
print(detect_langs("Otec matka syn."))

def same_lang(
    df
    ):
    df['same_lang'] = 0
    df = df.fillna('')
    
    for i in df.index:
        full_address_1 = detect(df.loc[i,'name_1'] + ' ' + df.loc[i,'address_1'] + ' ' + df.loc[i,'city_1'] + ' ' + df.loc[i,'state_1'] + ' ' + df.loc[i,'country_1'])
        full_address_2 = detect(df.loc[i,'name_2'] + ' ' + df.loc[i,'address_2'] + ' ' + df.loc[i,'city_2'] + ' ' + df.loc[i,'state_2'] + ' ' + df.loc[i,'country_2'])
        if (full_address_1 == full_address_2):
            df.loc[i,'same_lang'] = 1
            df.loc[i,'addr_lang_1'] = full_address_1
            df.loc[i,'addr_lang_2'] = full_address_2
            
    
    return df


th
ja
it
en
pl

[th:0.9999999994851496]
[ja:0.9999998799866179]
[so:0.9999985573962937]
[en:0.7142849560981814, it:0.28571435583988425]
[pl:0.8571394483837576, fi:0.14285787958897214]


In [11]:
df = pairs.head(10)
df = df.fillna('')
df['len_name_1'] = df['name_1'].str.len()
df['len_name_2'] = df['name_2'].str.len()
df['len_address_1'] = df['address_1'].str.len()
df['len_address_2'] = df['address_2'].str.len()
df['len_city_1'] = df['city_1'].str.len()
df['len_city_2'] = df['city_2'].str.len()
df['len_state_1'] = df['state_1'].str.len()
df['len_state_2'] = df['state_2'].str.len()
df['len_zip_1'] = df['zip_1'].str.len()
df['len_zip_2'] = df['zip_2'].str.len()
df['len_phone_1'] = df['phone_1'].str.len()
df['len_phone_2'] = df['phone_2'].str.len()


print(df)

                 id                           name   latitude   longitude  \
0  E_000001272c6c5d           Café Stad Oudenaarde  50.859975    3.634196   
1  E_000002eae2a589                 Carioca Manero -22.907225  -43.178244   
2  E_000007f24ebc95               ร้านตัดผมการาเกด  13.780813  100.484900   
3  E_000008a8ba4f48                       Turkcell  37.844510   27.844202   
4  E_00001d92066153        Restaurante Casa Cofiño  43.338196   -4.326821   
5  E_00002a131a2bf6              ministry of youth  29.364352   47.971362   
6  E_0000764d65557e                     McDonald's  -7.265894  112.749382   
7  E_00007dcd2bb53f              TOGO'S Sandwiches  38.257797 -122.064599   
8  E_0000890af22ff5  Flohmarkt Am Rathaus Steglitz  52.457045   13.322475   
9  E_0000c362229d93                     Coffee Cat   7.082218  125.610244   

                           address        city            state    zip  \
0                      Abdijstraat  Nederename  Oost-Vlaanderen   9700   
1   

In [24]:
df = pairs.head(10)
org_columns = df.columns
print(org_columns)
org_columns = org_columns.drop(['id','name'])
print(org_columns)
df['new_test'] = 0
print(df.columns)
df = df.drop(org_columns, axis = 1)
print(df)

Index(['id', 'name', 'latitude', 'longitude', 'address', 'city', 'state',
       'zip', 'country', 'url', 'phone', 'categories', 'point_of_interest'],
      dtype='object')
Index(['latitude', 'longitude', 'address', 'city', 'state', 'zip', 'country',
       'url', 'phone', 'categories', 'point_of_interest'],
      dtype='object')
Index(['id', 'name', 'latitude', 'longitude', 'address', 'city', 'state',
       'zip', 'country', 'url', 'phone', 'categories', 'point_of_interest',
       'new_test'],
      dtype='object')
                 id                           name  new_test
0  E_000001272c6c5d           Café Stad Oudenaarde         0
1  E_000002eae2a589                 Carioca Manero         0
2  E_000007f24ebc95               ร้านตัดผมการาเกด         0
3  E_000008a8ba4f48                       Turkcell         0
4  E_00001d92066153        Restaurante Casa Cofiño         0
5  E_00002a131a2bf6              ministry of youth         0
6  E_0000764d65557e                     McDonald'

/var/folders/sf/ctdwrxvj4p76kpw0svxdw30h0000gn/T/ipykernel_2260/1338131788.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['new_test'] = 0
